## 实现混淆矩阵，精准率和召回率

### 一、分类准确度的问题
一个癌症预测系统，输入体检信息，可以判断是否有癌症预测准确度：99.9%

如果癌症产生的概率只有 0.1%

**我们的系统预测所有人都是健康，即可达到 99.9%的准确率**
对于极度偏斜（Skewed Data）的数据，只使用分类准确度是远远不够的
#### 问题本质


### 二、混淆矩阵
![IMAGE](https://farm2.staticflickr.com/1826/42165477484_cc48b6cd26_o.png)
> 后面的字母代表预测值，比如FP，预测结果是 Postive，但是预测错了。应该是 Negeative，所以是 False Positive

![IMAGE](https://farm2.staticflickr.com/1769/42165509734_f6c2d46f87_o.png)
12: FP 本来没有病，我们判断他们有病
1: FN 本来有病，我们判断他没有病
8: TP 本来有病，我们判断他有病

### 三、精准率和召回率
精准率和召回率的分子都是相同的，都是 TP。

> 数字1（Positive）通常代表我们关注的时间，比如发病，有没有风险。

#### 1 精准率
![IMAGE](https://farm2.staticflickr.com/1789/41983696255_9400c4bed9_o.png)
我们做出20次预测，说明其中有人有病，有人没有患病，我们预测正确的概率是多少？

#### 2 召回率
![IMAGE](https://farm2.staticflickr.com/1724/41073939890_f66b93460a_o.png)
事件已经发生，实际上有10个癌症病人，我们能检验到其中多少人真正有病。


#### 对比
![IMAGE](https://farm2.staticflickr.com/1805/42835879492_fb940637cd_o.png)


#### 回顾之前的问题
![IMAGE](https://farm1.staticflickr.com/891/41074057490_39afc8e858_o.png)

In [1]:
import numpy as np
from sklearn import datasets

In [2]:
digits = datasets.load_digits()
X = digits.data
y = digits.target.copy()

# ❤️ 我们故意制造 skewed data
# 等于9的数字是1，不等于9的数据是0
y[digits.target==9] = 1
y[digits.target!=9] = 0

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666)

In [4]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
log_reg.score(X_test, y_test)

0.97555555555555551

In [6]:
y_log_predict = log_reg.predict(X_test)

In [7]:
def TN(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    return np.sum((y_true == 0) & (y_predict == 0))

TN(y_test, y_log_predict)

403

In [8]:
def FP(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    return np.sum((y_true == 0) & (y_predict == 1))

FP(y_test, y_log_predict)

2

In [9]:
def FN(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    return np.sum((y_true == 1) & (y_predict == 0))

FN(y_test, y_log_predict)

9

In [10]:
def TP(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    return np.sum((y_true == 1) & (y_predict == 1))

TP(y_test, y_log_predict)

36

In [11]:
def confusion_matrix(y_true, y_predict):
    return np.array([
        [TN(y_true, y_predict), FP(y_true, y_predict)],
        [FN(y_true, y_predict), TP(y_true, y_predict)]
    ])

confusion_matrix(y_test, y_log_predict)

array([[403,   2],
       [  9,  36]])

In [12]:
def precision_score(y_true, y_predict):
    tp = TP(y_true, y_predict)
    fp = FP(y_true, y_predict)
    try:
        return tp / (tp + fp)
    except:
        return 0.0
    
precision_score(y_test, y_log_predict)

0.94736842105263153

In [13]:
def recall_score(y_true, y_predict):
    tp = TP(y_true, y_predict)
    fn = FN(y_true, y_predict)
    try:
        return tp / (tp + fn)
    except:
        return 0.0
    
recall_score(y_test, y_log_predict)

0.80000000000000004

### scikit-learn中的混淆矩阵，精准率和召回率

In [14]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_log_predict)

array([[403,   2],
       [  9,  36]])

In [15]:
from sklearn.metrics import precision_score

precision_score(y_test, y_log_predict)

0.94736842105263153

In [16]:
from sklearn.metrics import recall_score

recall_score(y_test, y_log_predict)

0.80000000000000004